# Python Sockets

- https://docs.python.org/3/library/socket.html
- https://docs.python.org/3/library/internet.html

## Mitä on "Socket"?

"A network socket is a software structure within a network node of a computer network that serves as an endpoint for sending and receiving data across the network." ([Wikipedia](https://en.wikipedia.org/wiki/Network_socket))

Toisin sanottu Socket on käyttöjärjestelmän tarjoamaa rakennetta, joilla on mahdollista toteuttaa verkon yhteys toiselle koneelle ja sen kautta lähettää ja vastanottaa tietoa. Käytännössä yksi socket toteuttaa yksi TCP/UDP yhteys toisen koneen kanssa.

Huoma, että riippuu käyttöjärjestelmästä on olemassa erilaisia socket tyyppiä. Täällä puhutaan vain verkko -socketista, tunnetut myös Internet- tai BSD- socket nimillä.

## TCP- vs UDP- yhteydet

TCP yhteydet ovat luotettavaa tapaa lähettää tietoa. Protokolla varmistaa, että paketit saapuvat vastaanottajalle. Se lisää vähän kuormaa yhteydelle tähän tarkoitukseen ja siksi se on hitaampi protokolla.

UDP yhteydet ovat nopeaa tapaa lähettää tietoa. Se on nopea, mutta ei ole varmistusta, että paketit saapuvat vastaanottajalle.

## Miten socketit toimivat?

https://upload.wikimedia.org/wikipedia/commons/a/a1/InternetSocketBasicDiagram_zhtw.png

## Socket -moduuli

Moduuli tule Pythonin mukaan ja ei tarvitse asentaa sitä erikseen. Moduuli otetaan käyttöön tavallisesti:
```
import socket
```

Tärkeimmät moduulin funktiot/metoodit ovat:
- `socket()`
- `.bind()`
- `.listen()`
- `.accept()`
- `.connect()`
- `.send()`
- `.sendall()`
- `.recv()`
- `.close()`

Vähän tietoa jokaisesta:
- `socket.socket(family, type)`: Luo socket
    - `family`: AF_UNIX or AF_INET
    - `type`: SOCK_STREAM (TCP) or SOCK_DGRAM (UDP)
- `.bind((host, port))`: Ehdollista. Liittää socket tietyyn portiin. Ilman sitä käytetään random ja sopivaa porttia.
- `.listen()`: Määrittää, että socket on ns. "kuuntelija"
- `sock, r_addr = s.accept()`: Odottaa yhteyksiä
- `.connect((host, port))`: Ota aktiivisesti yhteyttä tietyyn host:port
- `.send()`: Lähettää bytes ( `str.encode()` ) (TCP)
- `.sendall()`: Samaa kuin `.send()`, mutta varmistaa, että kaikki tiedot on lähetetty
- `data = s.recv(max_length)`: Lue bytes välimuistista ( `data.decode()` ) (TCP)
- `.close()`: Close socket
UDP:lle:
- `.sento(databytes, address)`: Lähettää UDP packets
- `.recvfrom()`: Receives UDP packets
Lisää funktiota/metodia:
- `.gethostname()`: Get current hostname
- `.gethostbyname(name)`: Get IP of name
- `.getpeername()`: Remote address
- `.getsockname()`: Local endpoint


Echo Esimerkki: https://docs.python.org/3/library/socket.html#example

In [ ]:
# Echo server program
import socket


HOST = '127.0.0.1'        # Symbolic name meaning all available interfaces (''=0.0.0.0)
PORT = 50007              # Arbitrary non-privileged port (Ports >0 and <1024. Ports under 1024 need admin rights)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # socket.AF_INET = Internet socket
    # socket.SOCK_STREAM = TCP (socket.SOCK_DGRAM = UDP)
    s.bind((HOST, PORT))    # Socket will be bind to HOST:PORT
    s.listen(1)             # Set socket in listen mode
    conn, addr = s.accept() # Start to accept connections. When a connection is stablished, a socket, address pair is returned
    with conn:
        print('Connected by', addr)
        while True: # Read data until there is no more data
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)  # Send back the data. Hence the "Echo" -name of the program.

# The use of "with" is to avoid ".close()"

In [ ]:
# Echo client program
import socket


HOST = '127.0.0.1'        # The remote host
PORT = 50007              # The same port as used by the server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:        # Notice: Internet and TCP
    s.connect((HOST, PORT))     # Connect to remote port
    s.sendall(b'Hello, world')  # Send text. Notice that data is bytes
    data = s.recv(1024)         # Read back the echo

print('Received', repr(data))   # Print received data

Huoma ero, miten palvelin ja asiakas toimivat. Asiakas luo yksi socket, jolla ottaa yhteyttä ja lähettää ja vastaanottaa tietoa.
Palvelin luo socket, jolla "kuuntelee" yhteyksiä. Mahdolliset yhteydet muodostuvat uusilla socketilla.

Huoma myös, että jotkut funktiot/metodit, kuten `.accept` ja `.recv` pysäyttävät suoritusta siihen asti, että jotain tapahtuu (yhteys on muodostettu tai tietoa on saavuttu)

Suorittaa palvelin yhdessä terminaalissa ja asiakas toisessa terminaallissa
Kun palvelin on käynnissä, voidaan tarkasta yhteysien tilanne `netstat` -komennolla (`netstat -an` Windowsilla)

https://files.realpython.com/media/sockets-loopback-interface.44fa30c53c70.jpg (Local/sisäinen yhteys)
vs
https://files.realpython.com/media/sockets-ethernet-interface.aac312541af5.jpg (Verkko yhteys)


Miten käsitellään useita yhteyksiä samaan aikaan? -> "[Concurrency](https://www.geeksforgeeks.org/concurrency-in-operating-system/)":

- Multiprocess: [multitasking](https://en.wikipedia.org/wiki/Computer_multitasking). [IPC](https://en.wikipedia.org/wiki/Inter-process_communication). [Context switch](https://en.wikipedia.org/wiki/Context_switch). 
- Threading: Pythonissa single processor pre-emptive multitasking ([GIL](https://docs.python.org/3/glossary.html#term-global-interpreter-lock))
- AsyncIO: single-threaded cooperative multitasking
- Selectors: IO multiplexing

https://blog.floydhub.com/multiprocessing-vs-threading-in-python-what-every-data-scientist-needs-to-know/
https://stackoverflow.com/a/63519065 (multiprocessing vs multithreading vs asyncio in Python 3)
https://www.bryanbraun.com/2012/06/25/multitasking-and-context-switching/

# Threading

- https://docs.python.org/3/library/threading.html

"CPython implementation detail: In CPython, due to the Global Interpreter Lock, only one thread can execute Python code at once (even though certain performance-oriented libraries might overcome this limitation). If you want your application to make better use of the computational resources of multi-core machines, you are advised to use multiprocessing or concurrent.futures.ProcessPoolExecutor. However, threading is still an appropriate model if you want to run multiple I/O-bound tasks simultaneously."

- https://realpython.com/intro-to-python-threading/

```Python
From threading import Thread
```

Tarkeimmät funktiot:
- `t = Thread(target=func, args=())`: Luo thread
- `t.start()`: Käynnistää thread
- `t.join()`: Odottaa, että thread suorittuu loppuun asti


In [ ]:
from threading import Thread
import time

def testfunc(val):
    time.sleep(val)     # Kokeile 5-val
    print(val)

threads = []
for i in range(5):
    t = Thread(target=testfunc, args=(i,))
    threads.append(t)
    t.start()

for index, t in enumerate(threads):
    print(f'Joining thread {index}')
    t.join()

print("All other threads are finished")


Kun on iso määrä thread, mitä pitää luoda, voidaan käyttää `ThreadPoolExecutor`:

https://docs.python.org/3/library/concurrent.futures.html

```Python
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)
    print(future.result())

# Tai

with ThreadPoolExecutor(max_workers=n) as executor:
    executor.map(function_to_execute, iterable_first_arg, iterable_second_arg, ...)
```


In [ ]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))


In [ ]:
# Tosielämän esimerkki (Christian)

from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from jinja2 import Template 
from ping3 import ping
from time import sleep
import logging
import csv


def read_hosts(hostfile: str) -> list:
    """ Read hosts file """
    with open(hostfile, 'r') as file:
        csvreader = csv.reader(file)
        data = []
        for row in csvreader:
            data.append(row)
    return data

def create_host_matrix(data: list) -> dict:
    """ Prepare internal hosts data matrix from hosts list (from CSV) """
    hosts = {}
    for row in data:
        hosts[row[1]] = {
            'name'        : row[0],
            'ip'          : row[1],
            'online'      : False,
            'last_online' : 0,
            'state_change': 0
        }
    return hosts

def custom_ping(ip: str, online_previously: bool, timeout: str=1) -> bool:
    """ ping an IP address. Max 3 times (default timeout 1sec). Return True the first time host answers. 
        If host was previously offline, then returns result at first try. """
    times = 3 if online_previously else 1
    for _ in range(times):
        if ping(ip, timeout=timeout):
            return True
    return False

def ping_remote_host(host: dict) -> None:
    """ Ping remote host and update internal data based on the result """
    name = host['name']
    logging.debug(f'Thread {name}: Starting ping')
    ip = host['ip']
    original_state = host['online']
    if result := custom_ping(ip, original_state):
        host['last_online'] = datetime.now()
    host['online'] = result
    if original_state != host['online']:
        host['state_change'] = datetime.now()
    logging.debug(f'Thread {name}: Ping result: {result}')

if __name__ == "__main__":
    # Prepare logging
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")
    # logging.getLogger().setLevel(logging.DEBUG)

    # Prepare internal data structures
    logging.debug('Main: Read hosts and create internal data structures')
    hosts = create_host_matrix(read_hosts('hosts.csv'))
    hosts_ip = [ host['ip'] for host in hosts.values()]

    # Prepare HTML template
    logging.debug('Main: Prepare HTML template')
    with open('template.html','r') as f:
        tmpl = Template(''.join(f.readlines()))

    # Loop
    while True:
        logging.debug('Main: Launching threads')
        with ThreadPoolExecutor(max_workers=len(hosts)) as executor:
            executor.map(ping_remote_host, hosts.values())

        logging.debug('Main: Writing result to HTML file')
        with open('pingmon.html','w') as f:
            count = f.write(tmpl.render(hosts=hosts.values()))

        logging.debug('Main: sleeping 20sec')
        sleep(20)

Voidaan myös ajoittaa funktion suoritusta `Threading.Timer` -metodilla

```Python
from threading import Timer

t = Timer(delay_in_seconds, function_to_execute)
t.start()
```

In [ ]:
# Timer esimerkki
# Nähdään myös muuttujista

from datetime import datetime
import threading
from time import sleep
from random import randint


start = datetime.now()
globalvar = 0

def get_timedif():
    dif = datetime.now() - start
    return f'{dif.seconds // 60:02}:{dif.seconds % 60:02}'

def set_timer():
    global globalvar
    globalvar += 1
    localvar = globalvar    # localvar on paikallinen jokaiselle threadille
    print(f"[{get_timedif()}][{localvar:02}] - Setting timer")
    threading.Timer(5.0, set_timer).start()
    s = randint(5,10)
    print(f"[{get_timedif()}][{localvar:02}] - Executing: Sleeping for {s} seconds")
    sleep(s)
    print(f"[{get_timedif()}][{localvar:02}] - Exiting")
    

set_timer()

Server - Client esimerkki Thread -muodolla:

In [ ]:
# Echo server program
import socket
from threading import Thread

def receive_data(conn, addr):
    with conn:
        print('Connected by', addr)
        while True: # Read data until there is no more data
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)  # Send back the data. Hence the "Echo" -name of the program.


HOST = '127.0.0.1'        # Symbolic name meaning all available interfaces (''=0.0.0.0)
PORT = 50007              # Arbitrary non-privileged port (Ports >0 and <1024. Ports under 1024 need admin rights)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # socket.AF_INET = Internet socket
    # socket.SOCK_STREAM = TCP (socket.SOCK_DGRAM = UDP)
    s.bind((HOST, PORT))    # Socket will be bind to HOST:PORT
    s.listen(1)             # Set socket in listen mode
    while True:
        conn, addr = s.accept() # Start to accept connections. When a connection is stablished, a socket, address pair is returned
        Thread(target=receive_data, args=(conn, addr)).start()

# The use of "with" is to avoid ".close()"

In [ ]:
# Echo client program
import socket
from threading import Thread
from time import sleep

def make_connection(index):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:        # Notice: Internet and TCP
        s.connect((HOST, PORT))     # Connect to remote port
        s.sendall(bytes(f'Hello, world ({index})', 'utf-8'))  # Send text. Notice that data is bytes
        data = s.recv(1024)         # Read back the echo
        print('Received', repr(data))   # Print received data


HOST = '127.0.0.1'        # The remote host
PORT = 50007              # The same port as used by the server
Thread(target=make_connection, args=(1,)).start()
Thread(target=make_connection, args=(2,)).start()
sleep(1)

Tehtävä 1: Luo chatti asiakas, joka ota yhteyttä testi serveriin (172.20.1.50:65432). Sen jälkeen pitää printata kaikki, mitä vastaanottaa palvelimelta ja samaan aikaan se pysty kysymään käyttäjältä tekstiä, joka pitää lähettää

Tehtävä 2: Toteuttaa chatti serverin